<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/misc/Metric_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from collections import defaultdict
import torch
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

In [2]:
# Hyperparameter Section
BS = 64

In [5]:
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', 
                                             train=True, 
                                             transform=transform,
                                             download=True)
valid_dataset = torchvision.datasets.CIFAR10(root='/.data',
                                             train=False,
                                             transform=transform,
                                             download=True,
                                             )

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BS,
                          shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=BS,
                          shuffle=False)

Files already downloaded and verified



Extracting /.data/cifar-10-python.tar.gz to /.data


In [8]:
# Sanity check dataloader
for sample in train_loader:
  print(sample[0].shape)
  break

torch.Size([64, 3, 32, 32])


In [31]:
# Form a lookup
sample_train_loader = DataLoader(train_dataset,
                                 batch_size=1,
                                 shuffle=False,)
sample_test_loader = DataLoader(valid_dataset,
                                batch_size=1,
                                shuffle=False)

x_train = [sample[0].numpy() for sample in sample_train_loader]
y_train = [sample[1].item() for sample in sample_train_loader]

x_test = [sample[0].numpy() for sample in sample_test_loader]
y_test = [sample[1].item() for sample in sample_test_loader]

class_idx_to_train = defaultdict(list)
for train_idx, y in enumerate(y_train):
  class_idx_to_train[y].append(train_idx)

class_idx_to_test = defaultdict(list):
  for test_idx, y in enumerate(y_test):
    class_idx_to_test[y].append(test_idx)